In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score 
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.ticker as ticker
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri


In [2]:
robjects.r('devtools::install_github("josedv82/airball")')
robjects.r('library(airball)')

# Set the connection buffer size

buffer = '''
Sys.setenv(VROOM_CONNECTION_SIZE = 262144)
'''

robjects.r(buffer)

code = '''
nba_travel(start_season = 2020,
           end_season = 2022,
           team = c("Los Angeles Lakers"),
           return_home = 3,
           phase = "RS",
           flight_speed = 550)
'''
travel_data = robjects.r(code)
# travel_data
column_names = travel_data.colnames
print(column_names)

R[write to console]: Skipping install of 'airball' from a github remote, the SHA1 (a62db42b) has not changed since last install.
  Use `force = TRUE` to force installation

R[write to console]: The legacy packages maptools, rgdal, and rgeos, underpinning this package
will retire shortly. Please refer to R-spatial evolution reports on
https://r-spatial.org/r/2023/05/15/evolution4.html for details.
This package is now running under evolution status 0 



 [1] "Season"          "Phase"           "Month"           "Week"           
 [5] "Date"            "Team"            "Opponent"        "Location"       
 [9] "W/L"             "City"            "Distance"        "Route"          
[13] "Rest"            "TZ"              "Shift (hrs)"     "Flight Time"    
[17] "Direction (E/W)" "Return Home"     "Latitude"        "Longitude"      
[21] "d.Latitude"      "d.Longitude"    



In [3]:
pandas2ri.activate()

# Initialize an empty dictionary for storing the column data
pandas_dict = {}

# Iterate over each column
for name in column_names:
    # Extract the column data as a Python list
    column_data = list(travel_data.rx(name))
    # Convert any R vectors to Python lists
    column_data = [item[0] if isinstance(item, robjects.vectors.Vector) else item for item in column_data]
    # Add the column data to the dictionary
    pandas_dict[name] = column_data

# Create the Pandas DataFrame
travel_df = pd.DataFrame(pandas_dict)
travel_df

Season   
0  [2019-20, 2019-20, 2019-20, 2019-20, 2019-20, ...  \

                                               Phase   
0  [RS, RS, RS, RS, RS, RS, RS, RS, RS, RS, RS, R...  \

                                               Month   
0  [Oct, Oct, Oct, Oct, Nov, Nov, Nov, Nov, Nov, ...  \

                                                Week   
0  [43.0, 43.0, 43.0, 44.0, 44.0, 44.0, 45.0, 45....  \

                                                Date   
0  [18191.0, 18194.0, 18196.0, 18198.0, 18201.0, ...  \

                                                Team   
0  [Los Angeles Lakers, Los Angeles Lakers, Los A...  \

                                            Opponent   
0  [Los Angeles Clippers, Utah Jazz, Charlotte Ho...  \

                                            Location   
0  [Away, Home, Home, Home, Away, Away, Away, Hom...  \

                                                 W/L   
0  [L, W, W, W, W, W, W, W, L, W, W, W, W, W, W, ...  \

                                                City  ...   
0  [Los Angeles, Los Angeles, Los Angeles, Los An...  ...  \

                                                Rest   
0  [15.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0,...  \

                                                  TZ   
0  [America/Los_Angeles, America/Los_Angeles, Ame...  \

                                         Shift (hrs)   
0  [[0.0], [0.0], [0.0], [0.0], [2.0], [0.0], [0....  \

                                         Flight Time   
0  [-, -, -, -, ~2.27 hours, ~55 minutes, ~1.91 h...  \

                                     Direction (E/W)   
0  [-, -, -, -, East, -, -, West, -, East, West, ...  \

                                         Return Home   
0  [, , , , , , , , , , , , , , , , , , , , , , ,...  \

                                            Latitude   
0  [34.11, 34.11, 34.11, 34.11, 32.79, 29.46, 41....  \

                                           Longitude   
0  [-118.41, -118.41, -118.41, -118.41, -96.77, -...  \

                                          d.Latitude   
0  [34.11, 34.11, 34.11, 34.11, 34.11, 32.79, 29....  \

                                         d.Longitude  
0  [-118.41, -118.41, -118.41, -118.41, -118.41, ...  

[1 rows x 22 columns]

In [4]:
injuryData = pd.read_csv("~/Documents/Job Search - 2023/NBA_Injury_Predictor/Injuries.csv")

In [5]:
injuryData.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)
injuryData.rename(columns={'Notes': 'Injury Type'}, inplace=True)
injuryData.rename(columns={'Relinquished': 'Player'}, inplace=True)

### We want to separate the dataset by year to get a good idea of the total number of injuries that have occured annually

In [6]:
# Setting the objects in the column to a pandas datetime object
injuryData['Date'] = pd.to_datetime(injuryData['Date'])

#Grouping rows by 'decade' column
injuryData['Year'] = (injuryData['Date'].dt.year)



### Let's now remove all rows where the players have been acquired back from the IL 


In [7]:
# Removing the Acquired column removes all duplicates 
injuryData = injuryData[injuryData['Acquired'].isna()]
del injuryData['Acquired']

### Now let's calculate the total sum of injuries for each Year

## `It is important to note that the NBA did not have and IR/IL until the 1989-1990 season. Prior to this injured players were still members of the active roster. For this reason, it's likely many injuries do not appear in the dataset.`

In [8]:
# We only want to include data from the years after 1989 
yearly_injuries = {}
total_injuries = 0
for year, group in injuryData.groupby('Year'):
    if year > 1989: 
        injuries = group.shape[0]
        yearly_injuries[year] = injuries
        total_injuries += injuries
        
df = pd.DataFrame(yearly_injuries.items(), columns=['Year', 'Injuries'])
condition1 = df['Year'].astype(int) <= 2015
df.loc[condition1, 'Players'] = 354
df['isleagueExpansion'] = np.where(df['Year']<=2015, 0, 1)

# Set the condition for the second range of years (1996 onwards)
condition2 = df['Year'].astype(int) > 2015
df.loc[condition2, 'Players'] = 450
    

In [9]:
# adding the total number of games played in the NBA each year to the Dataframe
def get_games_played(df):

    games_played = {
      year: 6620
      for year in range(1990, 2024)
  }

  # Update the dictionary for years with a different number of games played.
    games_played[1999] = 4100
    games_played[2011] = 4096
    games_played[2020] = 4384
    games_played[2021] = 4384

  # Create a new column in the DataFrame for the number of games played.
    df['Games Played'] = df.Year.map(games_played)
    df['Games Played'] = df['Games Played'].astype(int)

get_games_played(df)

In [10]:
# Adding feature to account for a lockout season
df['Lockout'] = np.where(df['Year'].isin([1999, 2011]), 1, 0)

In [11]:
# Add feature for average number of miles travelled year over year
annual_travel = pd.read_csv("~/Documents/Job Search - 2023/NBA_Injury_Predictor/AverageTravel.csv")
df = pd.concat([df, annual_travel['Miles Traveled']], axis=1)

In [12]:
# why are there so few injuries in the early in the early 90s
# use a proxy to get data
# average NBA salary 


In [13]:
def normalize_players(df):
  # Get the mean and standard deviation of the players feature.
    mu = df['Players'].mean()
    std = df['Players'].std()

  # Normalize the players feature.
    df['Normalized Players'] = (df['Players'] - mu) / std
    
    return df

normalize_players(df)


Year  Injuries  Players  isleagueExpansion  Games Played  Lockout   
0   1990        64    354.0                  0          6620        0  \
1   1991        66    354.0                  0          6620        0   
2   1992        28    354.0                  0          6620        0   
3   1993        33    354.0                  0          6620        0   
4   1994       127    354.0                  0          6620        0   
5   1995       245    354.0                  0          6620        0   
6   1996       277    354.0                  0          6620        0   
7   1997       290    354.0                  0          6620        0   
8   1998       157    354.0                  0          6620        0   
9   1999       396    354.0                  0          4100        1   
10  2000       326    354.0                  0          6620        0   
11  2001       335    354.0                  0          6620        0   
12  2002       318    354.0                  0          6620        0   
13  2003       310    354.0                  0          6620        0   
14  2004       356    354.0                  0          6620        0   
15  2005       627    354.0                  0          6620        0   
16  2006       938    354.0                  0          6620        0   
17  2007       962    354.0                  0          6620        0   
18  2008       857    354.0                  0          6620        0   
19  2009       755    354.0                  0          6620        0   
20  2010       883    354.0                  0          6620        0   
21  2011       600    354.0                  0          4096        1   
22  2012       780    354.0                  0          6620        0   
23  2013       634    354.0                  0          6620        0   
24  2014       679    354.0                  0          6620        0   
25  2015       814    354.0                  0          6620        0   
26  2016       835    450.0                  1          6620        0   
27  2017      1172    450.0                  1          6620        0   
28  2018      1016    450.0                  1          6620        0   
29  2019       866    450.0                  1          6620        0   
30  2020       570    450.0                  1          4384        0   
31  2021      1582    450.0                  1          4384        0   
32  2022      1182    450.0                  1          6620        0   
33  2023       506    450.0                  1          6620        0   

    Miles Traveled  Normalized Players  
0          2611000           -0.546482  
1          2631760           -0.546482  
2          2652520           -0.546482  
3          2673280           -0.546482  
4          2694040           -0.546482  
5          2714800           -0.546482  
6          2735560           -0.546482  
7          2756320           -0.546482  
8          2777080           -0.546482  
9          2797840           -0.546482  
10         2818600           -0.546482  
11         2839360           -0.546482  
12         2860120           -0.546482  
13         2880880           -0.546482  
14         2901640           -0.546482  
15         2922400           -0.546482  
16         2943160           -0.546482  
17         2963920           -0.546482  
18         2984680           -0.546482  
19         3005440           -0.546482  
20         3026200           -0.546482  
21         3046960           -0.546482  
22         3067720           -0.546482  
23         3088480           -0.546482  
24         3109240           -0.546482  
25         3130000           -0.546482  
26         3150760            1.776066  
27         3171520            1.776066  
28         3192280            1.776066  
29         3213040            1.776066  
30         3233800            1.776066  
31         3254560            1.776066  
32         3275320            1.776066  
33         3296080            1.776066

In [14]:
# Attempting to use R package to get data for total number of miles travelled
reticulate.import_from_github("josedv82/airball")

NameError: name 'reticulate' is not defined

In [ ]:
# calculating the correlation for the two features I will use to train my model
Players_corr = df['Players'].corr(df['Injuries'])
Normalized_corr = df['Normalized Players'].corr(df['Injuries'])
Year_corr = df['Year'].corr(df['Injuries'])


print("Correlation for Players feature:", Players_corr)
print("Correlation for Year feature:", Year_corr)
print("Correlation for Players feature:", Normalized_corr)


In [ ]:
#Using Linear Regression function to fit our model
train_data = df[:26]

X_train = np.array(train_data[['Year', 'Normalized Players']]).reshape(-1, 2)
y_train = train_data['Injuries']
reg = LinearRegression()


fit_model = reg.fit(X_train, y_train)
print(df.shape)
X_train.shape


In [ ]:
# using a scikit-learn attribute, the coef represents the wieghts assigned to different features 
print(f"The Weight of feature(s): {fit_model.coef_}")

In [ ]:
# getting the intercept(bias) for our fitted model
print(f"The Bias of the model: {fit_model.intercept_}")

In [ ]:
# Adding our insample prediction for each year to our training data set
train_data.loc[:, 'prediction_insample'] = fit_model.predict(X_train).copy()
train_data.head()

In [ ]:
# Calculating the error, MSE, and ABS error for each year
train_data['error'] = train_data.prediction_insample - train_data.Injuries
train_data['squared_error'] = train_data['error']**2
train_data['abs'] = abs(train_data['error'])

train_data

In [ ]:
# I'm not sure why my error is zero 

# Returns the mean for error, squared error and absolute error for the training set
train_data[['error', 'squared_error', 'abs']].mean()

In [ ]:
sns.set(style="darkgrid")
sns.lineplot(x=train_data['Year'], y=train_data['Injuries'], data=train_data)
sns.lineplot(x=train_data['Year'], y=train_data['prediction_insample'], data=train_data)
plt.xlabel('Year')
plt.ylabel('Number of Injuries')
plt.title('Number of Injuries Year by Year')


### <u>Time to test on a new set of data:<u>

In [ ]:
test_data = df[26:]
 
X_test = np.array(test_data[['Year', 'Players']]).reshape([-1, 2])
y_test = test_data['Injuries']
X_test

In [ ]:
y_pred = fit_model.predict(X_test)
y_pred
# Adding our insample prediction for each year to our test data set
test_data.loc[:, 'prediction_insample'] = fit_model.predict(X_test).copy()
test_data.head()

In [ ]:
test_data['error'] = test_data.prediction_insample - test_data.Injuries
test_data['squared_error'] = test_data['error']**2
test_data['abs'] = abs(test_data['error'])

test_data

In [ ]:
df.describe()["Injuries"]

In [ ]:
plt.plot(X_test[:, 0], y_pred, 'r-', label='Predicted Data')
sns.set(style="darkgrid")
sns.lineplot(x=test_data['Year'], y=test_data['Injuries'], data=test_data)
plt.legend()
plt.xlabel('Year')
plt.ylabel('Number of Injuries')
plt.title('Number of Injuries Year by Year')
ax = plt.gca()
ax.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
plt.show()

In [ ]:
# Now let's remove all the rows from the data frame that have data before 1990
injuryData = injuryData[injuryData['Year'] >= 1990]